In [2]:
from generator import load_llama3_tokenizer
from datasets import load_dataset
from train import *
import json
import os
from tqdm import tqdm
from multiprocess import Pool
import itertools

def chunks(l, n):
    for i in range(0, len(l), n):
        yield (l[i: i + n], i // n)

def multiprocessing(strings, function, cores=6, returned=True):
    df_split = chunks(strings, len(strings) // cores)
    pool = Pool(cores)
    pooled = pool.map(function, df_split)
    pool.close()
    pool.join()

    if returned:
        return list(itertools.chain(*pooled))

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


[2025-05-11 13:18:48,021] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


In [7]:
mimi = None
device = 'cpu'
text_tokenizer = load_llama3_tokenizer()

dataset = load_dataset('mesolitica/Malaysian-Emilia-Sesame')['train']

README.md:   0%|          | 0.00/420 [00:00<?, ?B/s]

train-00000-of-00005.parquet:   0%|          | 0.00/73.8M [00:00<?, ?B/s]

train-00001-of-00005.parquet:   0%|          | 0.00/78.1M [00:00<?, ?B/s]

train-00002-of-00005.parquet:   0%|          | 0.00/82.6M [00:00<?, ?B/s]

train-00003-of-00005.parquet:   0%|          | 0.00/84.2M [00:00<?, ?B/s]

train-00004-of-00005.parquet:   0%|          | 0.00/84.4M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/4026870 [00:00<?, ? examples/s]

In [8]:
%%time

data = dataset[-1]
segment_tokens, segment_tokens_mask = get_input_ids(
    data['reference_audio'], 
    data['reference_text'],
    data['target_audio'], 
    data['target_text'],
    mimi=mimi,
    text_tokenizer=text_tokenizer,
    device=device,
    calculated_speech_tokens=True,
)
segment_tokens.shape[0]

CPU times: user 42.1 ms, sys: 19.6 ms, total: 61.7 ms
Wall time: 16.6 ms


308

In [15]:
def loop(indices):
    indices, _ = indices
    lengths = []
    
    dataset = load_dataset('mesolitica/Malaysian-Emilia-Sesame')['train']
    text_tokenizer = load_llama3_tokenizer()
    for i in tqdm(indices):
        data = dataset[i]
        segment_tokens, segment_tokens_mask = get_input_ids(
            data['reference_audio'], 
            data['reference_text'],
            data['target_audio'], 
            data['target_text'],
            mimi=mimi,
            text_tokenizer=text_tokenizer,
            device=device,
            calculated_speech_tokens=True,
        )
        lengths.append({
            'i': i,
            'length': segment_tokens.shape[0]
        })
    return lengths

In [16]:
lengths = loop((range(10), 0))

100%|██████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 625.10it/s]


In [17]:
lengths = multiprocessing(range(len(dataset)), loop, cores = 20)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

100%|██████████████████████████████████████████████████████████████████████████████| 201343/201343 [04:25<00:00, 757.29it/s]


In [18]:
rows = sorted(lengths, key = lambda x: x['length'])

In [19]:
maxlen = 1536
data, temp, l = [], [], 0
for r in tqdm(rows):
    l += r['length']
    temp.append(r['i'])
    if l >= maxlen:
        data.append(temp)
        temp = []
        l = 0

100%|█████████████████████████████████████████████████████████████████████████| 4026870/4026870 [00:05<00:00, 759054.10it/s]


In [21]:
len(data)

535207

In [22]:
import json

with open('merged.json', 'w') as fopen:
    json.dump(data, fopen)

In [25]:
from huggingface_hub import HfApi
api = HfApi()
api.upload_file(
    path_or_fileobj="merged.json",
    path_in_repo="merged.json",
    repo_id="mesolitica/Malaysian-Emilia-Sesame",
    repo_type="dataset",
)

merged.json:   0%|          | 0.00/36.2M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/mesolitica/Malaysian-Emilia-Sesame/commit/d8fdc5e773b089e0af780d1a367708a4199d5249', commit_message='Upload merged.json with huggingface_hub', commit_description='', oid='d8fdc5e773b089e0af780d1a367708a4199d5249', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/mesolitica/Malaysian-Emilia-Sesame', endpoint='https://huggingface.co', repo_type='dataset', repo_id='mesolitica/Malaysian-Emilia-Sesame'), pr_revision=None, pr_num=None)